In [14]:
# default imports
import numpy as np
import pandas as pd
import os, sys
import re
import matplotlib.pyplot as plt
import pickle
# ChromAn related imports
sys.path.append(r'/lab/weissman_imaging/puzheng/Softwares/') # parent folder of ChromAn
#import torch
print(os.getpid())

3795443


In [10]:
from ChromAn.src.default_parameters import default_channels

In [11]:
default_channels


['750', '647', '561', '488', '405']

In [4]:
import ImageAnalysis3 as ia
%matplotlib notebook

from ImageAnalysis3 import *


In [ ]:
source_folder = r'\\10.245.74.158\Chromatin_NAS_0\Corrections\20210514-Corrections_2color_50_new_fiber'
target_folder = r'\\10.245.74.158\Chromatin_NAS_0\Corrections\20210514-Corrections_2color_40_new_fiber_from_50'
if not os.path.exists(target_folder):
    print(f"create folder: {target_folder}")
    os.makedirs(target_folder)
    
src_shape = [50,2048,2048]
tar_shape = [40,2048,2048]

In [5]:
correction_master_folder = r"/lab/weissman_imaging/puzheng/Corrections/"
source_folder = os.path.join(correction_master_folder, '20230902-Merscope01_s30_n500')

print(f"- source folder: {source_folder}")
# generate target folder
target_z = 40
step_size = 500
target_folder = os.path.join(correction_master_folder, f'20240107-Merscope01_s{target_z}_n{step_size}')
print(f"- target folder: {target_folder}")
if not os.path.exists(target_folder):
    print(f"-- creating target folder: {target_folder}")
    os.makedirs(target_folder)
    

- source folder: /lab/weissman_imaging/puzheng/Corrections/20230902-Merscope01_s30_n500
- target folder: /lab/weissman_imaging/puzheng/Corrections/20240107-Merscope01_s40_n500
-- creating target folder: /lab/weissman_imaging/puzheng/Corrections/20240107-Merscope01_s40_n500


In [7]:
src_shape = [30,2304,2304]
tar_shape = [40,2304,2304]

# illumination corr

In [9]:
channels

['750', '647', '561', '488', '405']

In [12]:
from shutil import copyfile

file_prefix = 'illumination_correction_'
channels = ['748','637','545','477','405']

for _ch in channels:
    _src_fl = os.path.join(source_folder, 
                           file_prefix+f"{_ch}_{src_shape[-2]}x{src_shape[-1]}.npy")
    _tar_fl = os.path.join(target_folder, 
                           file_prefix+f"{_ch}_{tar_shape[-2]}x{tar_shape[-1]}.npy")
    if os.path.isfile(_src_fl):
        print(_ch)
        copyfile(_src_fl, _tar_fl)

748
637
545
477
405


# chromatic abbrevation corr

In [17]:
from ChromAn.src.correction_tools.chromatic import generate_polynomial_data

In [18]:
# load chromatic abbrevation pf
file_prefix = 'chromatic_correction_'
ref_ch = '637'
chromatic_corr_channels = ['748', '545']
for _ch in chromatic_corr_channels:
    _basename = file_prefix+f"{_ch}_{ref_ch}_{src_shape[-2]}x{src_shape[-1]}.npy"
    _src_fl = os.path.join(source_folder, 
                           file_prefix+f"{_ch}_{ref_ch}_{src_shape[-3]}_{src_shape[-2]}_{src_shape[-1]}_const.pkl")
    _tar_fl = os.path.join(target_folder, 
                           file_prefix+f"{_ch}_{ref_ch}_{tar_shape[-3]}_{tar_shape[-2]}_{tar_shape[-1]}_const.pkl")
    
    if os.path.isfile(_src_fl):
        print(_ch)
        _info = pickle.load(open(_src_fl, 'rb'))
        # generate new chromatic const file
        new_info = {_k:_v for _k,_v in _info.items()}
        new_info['ref_center'] = (np.array(tar_shape)/2)
        # save new info
        pickle.dump(new_info, open(_tar_fl, 'wb'))
        print(_tar_fl)
        ## generate profiles
        # generate pixels
        _pixel_coords = np.indices(tar_shape)
        _pixel_coords = _pixel_coords.reshape(np.shape(_pixel_coords)[0], -1)
        _pixel_coords = _pixel_coords - (np.array(tar_shape)/2)[:, np.newaxis]
        #from ImageAnalysis3.correction_tools.chromatic import generate_polynomial_data
        # generate profile
        _profile = []
        for _i, _order in enumerate(_info['fitting_orders']):
            _pX = generate_polynomial_data(_pixel_coords.transpose(), _order)
            _py = np.dot(_pX, _info['constants'][_i]).reshape(tar_shape)
            _profile.append(_py)
        _profile = np.array(_profile)
        # save profile
        np.save(_tar_fl.replace("_const.pkl", ""), _profile)

748
/lab/weissman_imaging/puzheng/Corrections/20240107-Merscope01_s40_n500/chromatic_correction_748_637_40_2304_2304_const.pkl
545
/lab/weissman_imaging/puzheng/Corrections/20240107-Merscope01_s40_n500/chromatic_correction_545_637_40_2304_2304_const.pkl


# bleedthrough corr

In [19]:
# load chromatic abbrevation pf
file_prefix = 'bleedthrough_correction_'
bleed_corr_channels = ['748', '637','545']
#750_647_561_2048_2048
_basename = file_prefix
for _ch in bleed_corr_channels:
    _basename += f"{_ch}_"
_basename += f"{src_shape[-2]}_{src_shape[-1]}.npy"
_src_fl = os.path.join(source_folder, _basename)
_pf = np.load(_src_fl)

In [24]:
_tar_fl = os.path.join(target_folder, _basename)
print(_tar_fl)
np.save(_tar_fl.split('.npy')[0], _pf)

/lab/weissman_imaging/puzheng/Corrections/20240107-Merscope01_s40_n500/bleedthrough_correction_748_637_545_2304_2304.npy
